In [16]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

In [17]:
function kalman_filter_graph()
    x_prev_add = AdditionNode()
    
    x_prior = datavar(:x_prior, Normal{Float64})
    add_1   = constvar(:add_1, 1.0)

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = constvar(:noise, Normal(0.0, sqrt(200.0)))
    
    add_x_and_noise = AdditionNode()
    
    x = simplerandomvar(:x)
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = datavar(:y, Float64)
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(x_prev_add)
    activate!(add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()
    
    beliefs = Vector{Normal{Float64}}(undef, N)
    updates = Channel{Tuple{Normal{Float64}, Float64}}(Inf) do ch
        while true
            update_data = take!(ch)
            update!(x_prior, update_data[1])
            update!(y, update_data[2])
        end
    end
    
    actor = sync(lambda(Tuple{Int, AbstractBelief},
        on_next = (d) -> begin
        
            index  = d[1]    
            belief = getdata(d[2])
                    
            if index < N
                push!(updates, (belief, data[index + 1]))
            else
                finish!(x_prior)
                finish!(y)
                close(updates)
            end
                
            beliefs[index] = belief
        end,
        on_error = (e) -> println(e)
    ), timeout = 10000)
    
    @async begin
        try
            subscribe!(belief(x) |> enumerate(), actor)
            push!(updates, (Normal(0.0, 1000.0), data[1]))
        catch err 
            println(err)
        end
    end
    
    wait(actor)
    
    return beliefs
end

kalman (generic function with 1 method)

In [18]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [21]:
@btime kalman($data)

  1.055 ms (13673 allocations: 583.13 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-6.755880377703271, σ=14.140721622265264)
 Normal{Float64}(μ=-12.975589236664225, σ=9.999500037496876)
 Normal{Float64}(μ=-1.4689529628026596, σ=8.164693657357805)
 Normal{Float64}(μ=1.3373576233796287, σ=7.070891041799029)
 Normal{Float64}(μ=1.756603754186064, σ=6.3244288330249585)
 Normal{Float64}(μ=7.688710277324968, σ=5.773406469256952)
 Normal{Float64}(μ=7.943458596456585, σ=5.34514847952991)
 Normal{Float64}(μ=12.267168768371933, σ=4.999937501171851)
 Normal{Float64}(μ=10.957090424582162, σ=4.713992830503184)
 Normal{Float64}(μ=13.588886690774183, σ=4.472091234310838)
 Normal{Float64}(μ=14.402608767342823, σ=4.263975563874187)
 Normal{Float64}(μ=14.313202756984134, σ=4.082448884373011)
 Normal{Float64}(μ=13.666556944038334, σ=3.922292531398713)
 ⋮
 Normal{Float64}(μ=589.0307792219833, σ=0.5827164478148905)
 Normal{Float64}(μ=590.0272343592519, σ=0.5822224110578259)
 Normal{Float64}(μ=590.9972077984941, σ=0.5817296287317493)

In [20]:
@time kalman(data)

  0.001958 seconds (13.67 k allocations: 583.109 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-6.755880377703271, σ=14.140721622265264)
 Normal{Float64}(μ=-12.975589236664225, σ=9.999500037496876)
 Normal{Float64}(μ=-1.4689529628026596, σ=8.164693657357805)
 Normal{Float64}(μ=1.3373576233796287, σ=7.070891041799029)
 Normal{Float64}(μ=1.756603754186064, σ=6.3244288330249585)
 Normal{Float64}(μ=7.688710277324968, σ=5.773406469256952)
 Normal{Float64}(μ=7.943458596456585, σ=5.34514847952991)
 Normal{Float64}(μ=12.267168768371933, σ=4.999937501171851)
 Normal{Float64}(μ=10.957090424582162, σ=4.713992830503184)
 Normal{Float64}(μ=13.588886690774183, σ=4.472091234310838)
 Normal{Float64}(μ=14.402608767342823, σ=4.263975563874187)
 Normal{Float64}(μ=14.313202756984134, σ=4.082448884373011)
 Normal{Float64}(μ=13.666556944038334, σ=3.922292531398713)
 ⋮
 Normal{Float64}(μ=589.0307792219833, σ=0.5827164478148905)
 Normal{Float64}(μ=590.0272343592519, σ=0.5822224110578259)
 Normal{Float64}(μ=590.9972077984941, σ=0.5817296287317493)

In [19]:
@time kalman(data)

  0.469831 seconds (1.22 M allocations: 63.602 MiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-6.755880377703271, σ=14.140721622265264)
 Normal{Float64}(μ=-12.975589236664225, σ=9.999500037496876)
 Normal{Float64}(μ=-1.4689529628026596, σ=8.164693657357805)
 Normal{Float64}(μ=1.3373576233796287, σ=7.070891041799029)
 Normal{Float64}(μ=1.756603754186064, σ=6.3244288330249585)
 Normal{Float64}(μ=7.688710277324968, σ=5.773406469256952)
 Normal{Float64}(μ=7.943458596456585, σ=5.34514847952991)
 Normal{Float64}(μ=12.267168768371933, σ=4.999937501171851)
 Normal{Float64}(μ=10.957090424582162, σ=4.713992830503184)
 Normal{Float64}(μ=13.588886690774183, σ=4.472091234310838)
 Normal{Float64}(μ=14.402608767342823, σ=4.263975563874187)
 Normal{Float64}(μ=14.313202756984134, σ=4.082448884373011)
 Normal{Float64}(μ=13.666556944038334, σ=3.922292531398713)
 ⋮
 Normal{Float64}(μ=589.0307792219833, σ=0.5827164478148905)
 Normal{Float64}(μ=590.0272343592519, σ=0.5822224110578259)
 Normal{Float64}(μ=590.9972077984941, σ=0.5817296287317493)